<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FYP Demonstration
### Will Fitch 1633241 
### Brunel University Department of Computer Science

This notebook is a demonstration of my final year project, in which I have used neural networks and transfer learning to infer greenhouse gas information from satellite data.

All notebooks in this project are to be considered development environments, rather than bona fide scripts that, when run, will produce the end product. Therefore, certain code blocks and documentation are added for developer convenience

## Setup 

#### Imports

In [ ]:
# Sometimes fastai doesn't want to play with colab, so we remove it and replace 
# it with a compliant version just to be sure. 
!pip uninstall -y fastai
!pip install -U --no-cache-dir fastai

%cd /content
!git clone https://github.com/WRFitch/fyp.git

In [ ]:
import ee
import folium
import os
import time

from fastai.vision.all import *
from geopy.geocoders import Nominatim
from google.colab import drive
from IPython.display import Image
from osgeo import gdal
from PIL import Image
from pprint import pprint

ee.Authenticate()
ee.Initialize()
drive.mount('/content/drive')

# Import fyputil library
# EE stuff needs initialising first
%cd /content/fyp/src/fyputil
import constants as c
import ee_constants as eec
import ee_utils as eeutil
import fyp_utils as fyputil
%cd /content

#### Parts that don't require human input 

In [3]:
# Initialise datasets
# TODO update these for portability 
demo_dir = c.demo_dir
ghg_df = pd.read_csv(c.ghg_csv)
norm_ghg_df = fyputil.normGhgDf(ghg_df)
# initialise nominatim client so we can get coords from postcode
nominatim = Nominatim(user_agent="tutorial") # TODO look up whether this user agent is fine. 

def getGhgsAsArr(img_path):
  return fyputil.getGhgsAsArr(img_path, ghg_df)

model = load_learner(f"{c.model_dir}/{c.model_name}.pkl")

In [33]:
# Parses postcode into coordinates. You can put in anything else, like "kenya"
def getCoordsFromString(s):
  location = nominatim.geocode(s).raw
  return (float(location['lon']), float(location['lat']))

def getGhgsFromCoords(coords):
  img_path = importImg(coords)
  print(img_path)
  ghgs = getGhgsFromImg(img_path)
  displayGhgs(ghgs)
  displayHealthDefects(ghgs)
  displayEnvEffects(ghgs)

# use image pipeline to download an image based on a point object defined by the 
# given coordinates
def importImg(coords):
  # get a square from the given image. 
  # TODO why is this buffer 500? the ideal pixel size should be 224, so 2240m. 
  polygon = ee.Geometry.Point(coords).buffer(700).bounds()

  name = f"{coords[0]}_{coords[1]}"
  tifname = f"{name}.tif"

  # check if image is cached before importing from GEE
  cachePath = f"{c.demo_dir}/{name}.png"
  if os.path.isfile(cachePath): 
    print("image for these coordinates has already been imported!")
    return cachePath

  tifpath = f"{demo_path}/{tifname}"
  if not os.path.isfile(tifpath):
    print(f"importing {tifname}")
    eeutil.exportGeotiff(eec.s2_img, polygon, 10, "demo_export", name)
    
    while not os.path.isfile(tifpath):
      time.sleep(5)
    print(f"imported  {tifname}")
  
  # Convert GeoTIFF to PNG.
  fyputil.geotiffToPng(c.demo_dir, png_path=c.demo_dir)
  fyputil.rmArtifact(f"{demo_path}/{name}.png.aux.xml")
  print(f"{tifpath} converted to PNG")

  importPath = f"{demo_path}/{name}.png"
  if os.path.isfile(importPath): return importPath

  print("image import failed")

def getGhgsFromImg(img_path):
  print("getting GHG concentrations from given image")
  display(Image.open(img_path))
  predictions = model.predict(img_path)[0]
  dnorm_preds = fyputil.deNormPrediction(predictions, ghg_df)
  print(predictions)
  return predictions 

def displayGhgs(ghgs):
  print(ghgs)
  # Add ghgs to table
  # compare against average
  # display ghgs as a series of plots. 
  # for each ghg concentration, display health defects and mitigation strategies 
  # on table
  # print table

def displayHealthDefects(ghgs):
  print(ghgs)
  # for each ghg, compare against concentration. Depending on how high they are,
  # display the potential health effects and their likelihoods. 
  # return dict of ghg to string tuples of horrible effects and their likelihoods. 

def displayEnvEffects(ghgs):
  print(ghgs)
  # for each ghg, compare against concentration. Depending on how high they are, 
  # display the potential environmental effects and their likelihoods. 
  # return dict of ghg to string tuples of horrible effects and their likelihoods. 

## Demonstration

### Visualisation 

In [5]:
# TODO add error heatmaps
eec.map 

### Model Interpolation

Replace the "postcode" string with any string you like that pertains to a real geographic place; Nominatim can parse basically any location, since it uses OpenStreetMap. 

In [6]:
demo_coords = c.brunel_coords

In [12]:
postcode = "UB8 3PH"
demo_coords = getCoordsFromString(postcode)

In [ ]:
getGhgsFromCoords(demo_coords)

In [ ]:
getGhgsFromCoords(getCoordsFromString("buckingham palace"))

# Brunel Analysis

In [ ]:
brunel_stats = model.predict(f"{c.demo_dir}/-0.47278354461716354_51.53325658151181.png")
bpred = fyputil.deNormPrediction(brunel_stats[0], ghg_df)

In [ ]:
# TODO update to provide the 
for i in range(0, 6):
  print(dnorm_ghg_df[c.ghg_bands[i]].quantile(0.5))